## 0. Libraries import

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# imports 
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
#train test split
from sklearn.model_selection import train_test_split
# Hyperparameters selection
from sklearn.model_selection import RandomizedSearchCV
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
# models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
# error
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

## 1. Data import

### 1.1 Impor train data

In [3]:
df_diamonds_train=pd.read_csv('../data/diamonds_train.csv')
df_diamonds_train.pop("Unnamed: 0") #dropped an unnecessary column
df_diamonds_train

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,Very Good,H,VS2,Kimberly
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,2686,0.71,Fair,G,VS1,Las Vegas
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,Good,D,SI1,Kimberly
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,Ideal,G,SI1,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,10070,1.34,Ideal,G,VS1,Antwerp
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,12615,2.02,Good,F,SI2,Madrid
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01,5457,1.01,Ideal,H,SI1,Kimberly
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76,456,0.33,Ideal,J,VS1,Kimberly


### 1.2 Impor test data

In [4]:
df_diamonds_test=pd.read_csv('../data/diamonds_test.csv')
df_diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


## 2. Data preparation (training set)

### 2.1 Defining numerical and categorical features

In [5]:
# Excluding city (with trees sometimes is better to keep variables even if they are correlated)
num_features_list=['x','y','z','depth','table','carat']
cat_features_list=['cut','color','clarity']
target='price'
features_list=['x','y','z','depth','table','carat','cut','color','clarity']

### 2.2 Checking if zero/null values

In [6]:
df_diamonds_train[num_features_list].isna().sum()

x        0
y        0
z        0
depth    0
table    0
carat    0
dtype: int64

In [7]:
if 0 in df_diamonds_train[num_features_list]:
    print('0 values')
else:
    print('No 0 values')

No 0 values


In [8]:
df_diamonds_train['x'].min()

0.0

### 2.3 Remove outliers

In [9]:
def remove_outliers(df,feature):
    # IQR
    Q1 = np.percentile(df[feature], 25,
                   interpolation = 'midpoint')
    Q3 = np.percentile(df[feature], 75,
                   interpolation = 'midpoint')
    IQR = Q3 - Q1
    # Upper and lower
    upper_limit=Q3+1.5*IQR
    lower_limit=Q1-1.5*IQR
    # Removing the Outliers
    return df[(df[feature]>=lower_limit) & (df[feature]<=upper_limit)]

In [10]:
df_diamonds_train=remove_outliers(df_diamonds_train,'x')
df_diamonds_train=remove_outliers(df_diamonds_train,'y')
df_diamonds_train=remove_outliers(df_diamonds_train,'z')
df_diamonds_train=remove_outliers(df_diamonds_train,'depth')
df_diamonds_train=remove_outliers(df_diamonds_train,'table')
df_diamonds_train=remove_outliers(df_diamonds_train,'carat')

In [11]:
df_diamonds_train.shape

(37264, 12)

### 3. Feature engineering (training set)

### 3.1 Target encoding (mean and std)

In [12]:
# 1.Target encoding for categorical variables
# Mean
cut_encoding = df_diamonds_train.groupby(['cut'])['price'].mean().to_dict()
df_diamonds_train['cut_encoding'] = df_diamonds_train['cut'].map(cut_encoding).astype(float)
color_encoding = df_diamonds_train.groupby(['color'])['price'].mean().to_dict()
df_diamonds_train['color_encoding'] = df_diamonds_train['color'].map(color_encoding).astype(float)
clarity_encoding = df_diamonds_train.groupby(['clarity'])['price'].mean().to_dict()
df_diamonds_train['clarity_encoding'] = df_diamonds_train['clarity'].map(clarity_encoding).astype(float)
# Std
cut_encoding_std = df_diamonds_train.groupby(['cut'])['price'].std().to_dict()
df_diamonds_train['cut_encoding_std'] = df_diamonds_train['cut'].map(cut_encoding_std).astype(float)
color_encoding_std = df_diamonds_train.groupby(['color'])['price'].std().to_dict()
df_diamonds_train['color_encoding_std'] = df_diamonds_train['color'].map(color_encoding_std).astype(float)
clarity_encoding_std = df_diamonds_train.groupby(['clarity'])['price'].std().to_dict()
df_diamonds_train['clarity_encoding_std'] = df_diamonds_train['clarity'].map(clarity_encoding_std).astype(float)

In [13]:
df_diamonds_train.head()

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city,cut_encoding,color_encoding,clarity_encoding,cut_encoding_std,color_encoding_std,clarity_encoding_std
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai,4090.451413,4205.372106,3706.916046,3786.059445,3203.931679,3773.320777
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,Very Good,H,VS2,Kimberly,3716.148388,3989.238859,3706.916046,3552.522893,3617.921017,3773.320777
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,Good,D,SI1,Kimberly,3629.673247,3032.494785,3667.767250,3352.922564,3210.147580,3345.026353
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,Ideal,G,SI1,Dubai,3209.761477,3813.444988,3667.767250,3433.146709,3821.613156,3345.026353
5,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,61.2,57.0,7.45,7.39,4.54,9057,1.52,Ideal,F,SI2,Tel Aviv,3209.761477,3553.853659,4133.827826,3433.146709,3603.654261,3073.028701


### 3.2 Including volume

In [14]:
#df_diamonds_train['volume']=df_diamonds_train['x']*df_diamonds_train['y']*df_diamonds_train['z']

### 3.3 Cross Target encoding (mean and std)

In [15]:
# 2. Cross target encoding
# Mean
cut_color_encoding = df_diamonds_train.groupby(['cut','color'])['price'].mean().to_dict()
df_diamonds_train['cut_color_encoding'] = df_diamonds_train.set_index(['cut',
                                                                       'color']).index.map(cut_color_encoding.get).astype(float)
cut_clarity_encoding = df_diamonds_train.groupby(['cut','clarity'])['price'].mean().to_dict()
df_diamonds_train['cut_clarity_encoding'] = df_diamonds_train.set_index(['cut',
                                                                       'clarity']).index.map(cut_clarity_encoding.get).astype(float)
color_clarity_encoding = df_diamonds_train.groupby(['color','clarity'])['price'].mean().to_dict()
df_diamonds_train['color_clarity_encoding'] = df_diamonds_train.set_index(['color',
                                                                          'clarity']).index.map(color_clarity_encoding.get).astype(float)
# Std  
#cut_color_encoding_std = df_diamonds_train.groupby(['cut','color'])['price'].std().to_dict()
#df_diamonds_train['cut_color_encoding_std'] = df_diamonds_train.set_index(['cut',
#                                                                       'color']).index.map(cut_color_encoding_std.get).astype(float)
#cut_clarity_encoding_std = df_diamonds_train.groupby(['cut','clarity'])['price'].std().to_dict()
#df_diamonds_train['cut_clarity_encoding_std'] = df_diamonds_train.set_index(['cut',
#                                                                       'clarity']).index.map(cut_clarity_encoding_std.get).astype(float)
#color_clarity_encoding_std = df_diamonds_train.groupby(['color','clarity'])['price'].std().to_dict()
#df_diamonds_train['color_clarity_encoding_std'] = df_diamonds_train.set_index(['color',
#                                                                           'clarity']).index.map(color_clarity_encoding_std.get).astype(float)


In [16]:
features_list_encoding=['x','y','z','depth','table','carat','cut','color','clarity','cut_encoding','color_encoding'
                        ,'clarity_encoding','cut_encoding_std','color_encoding_std','clarity_encoding_std'
                        ,'cut_color_encoding','cut_clarity_encoding','color_clarity_encoding'
#                       ,'cut_color_encoding_std','cut_clarity_encoding_std','color_clarity_encoding_std'
                       ]

In [17]:
#num_features_encoding=['x','y','z','depth','table','carat','cut_encoding','color_encoding','clarity_encoding',
#                        'cut_encoding_std','color_encoding_std','clarity_encoding_std','volume','cut_color_encoding',
#                        'cut_clarity_encoding','color_clarity_encoding','cut_color_encoding_std',
#                        'cut_clarity_encoding_std','color_clarity_encoding_std']
#target='price'

### 3.4 Feature Scaling

With few exceptions, Machine Learning algrorithms don't perform well when the input numerical attributes have very different scales. We sure want our models to work well, so how can we go about it?

Feature scaling can be done in 2 ways: Min-max scaling and Standardization. I would preferably use Standardization, because it is much less affected by outliers. Scikit-Learn provides a transformer called StandardScaler for this transformation.

In [18]:
#Nodf_diamonds_train_num=df_diamonds_train[num_features_encoding]
#df_diamonds_train_num=df_diamonds_train[num_features_list]
#df_diamonds_train_carat=df_diamonds_train[cat_features_list]
#df_diamonds_train_target=df_diamonds_train['price']

In [19]:
# Perform the feature scaling on the numeric attributes of the dataset
#num_scaler = StandardScaler()
#df_diamonds_train_num_scaled = num_scaler.fit_transform(df_diamonds_train_num)
#df_diamonds_train_num_scaled
#df_diamonds_train_num_scaled=pd.DataFrame(df_diamonds_train_num_scaled)
#NOdf_diamonds_train_num_scaled.rename(columns = {0:'x', 1:'y', 2:'z', 3:'depth', 4:'table', 5:'carat', 6:'cut_encoding',
#                                              7:'color_encoding', 8:'clarity_encoding', 9:'cut_encoding_std',
#                                               10:'color_encoding_std',11:'clarity_encoding_std', 12:'volume',
#                                               13:'cut_color_encoding',14:'cut_clarity_encoding',
#                                               15:'color_clarity_encoding',16:'cut_color_encoding_std',
#                                               17:'cut_clarity_encoding_std',18:'color_clarity_encoding_std'}, 
#                           inplace = True)
#df_diamonds_train_num_scaled.rename(columns = {0:'x', 1:'y', 2:'z', 3:'depth', 4:'table', 5:'carat'}, 
#                           inplace = True)
#df_diamonds_train_num_scaled.head()

In [20]:
#df_diamonds_train=pd.merge(df_diamonds_train_num_scaled, df_diamonds_train_carat, left_index=True, right_index=True)
#df_diamonds_train=pd.merge(df_diamonds_train, df_diamonds_train_target, left_index=True, right_index=True)
#df_diamonds_train.head()

### 3.5 One hot encoding

In [21]:
# 3. Defining features y target
#features_list_volume=['x', 'y', 'z', 'depth', 'table', 'carat', 'cut', 'color', 'clarity','volume']
X=df_diamonds_train[features_list_encoding]
y=df_diamonds_train['price']

In [22]:
# 4.One-hot encoding for categorical variables
X=pd.get_dummies(X,columns=cat_features_list,drop_first=True)

In [23]:
X.head()

,x,y,z,depth,table,carat,cut_encoding,color_encoding,clarity_encoding,cut_encoding_std,color_encoding_std,clarity_encoding_std,cut_color_encoding,cut_clarity_encoding,color_clarity_encoding,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,6.83,6.79,4.25,62.4,58.0,1.21,4090.451413,4205.372106,3706.916046,3786.059445,3203.931679,3773.320777,4723.032323,4279.325321,4072.978678,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,4.35,4.38,2.75,63.0,57.0,0.32,3716.148388,3989.238859,3706.916046,3552.522893,3617.921017,3773.320777,4131.209839,4064.356420,4536.957035,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
3,4.68,4.72,3.00,63.8,56.0,0.41,3629.673247,3032.494785,3667.767250,3352.922564,3210.147580,3345.026353,3092.344221,3398.128440,2890.454856,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,6.55,6.51,3.95,60.5,59.0,1.02,3209.761477,3813.444988,3667.767250,3433.146709,3821.613156,3345.026353,3622.447529,3442.506284,3608.093343,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
5,7.45,7.39,4.54,61.2,57.0,1.52,3209.761477,3553.853659,4133.827826,3433.146709,3603.654261,3073.028701,3214.139157,3850.286193,3891.265835,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


### 3.6 Define train and validation

In [24]:
# 5.Splitting train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
X_train.shape

(29811, 32)

In [26]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29811 entries, 11656 to 17132
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   x                       29811 non-null  float64
 1   y                       29811 non-null  float64
 2   z                       29811 non-null  float64
 3   depth                   29811 non-null  float64
 4   table                   29811 non-null  float64
 5   carat                   29811 non-null  float64
 6   cut_encoding            29811 non-null  float64
 7   color_encoding          29811 non-null  float64
 8   clarity_encoding        29811 non-null  float64
 9   cut_encoding_std        29811 non-null  float64
 10  color_encoding_std      29811 non-null  float64
 11  clarity_encoding_std    29811 non-null  float64
 12  cut_color_encoding      29811 non-null  float64
 13  cut_clarity_encoding    29811 non-null  float64
 14  color_clarity_encoding  29811 non-

## 4. Data preparation (test set)

## 5. Feature engineering (test set) 

In [27]:
# 0. Adapting categorical features for validation model
X_test=df_diamonds_test[features_list]

### 5.1 Target encoding (mean and std)

In [28]:
# 1.Target encoding for categorical variables
# Mean
cut_encoding = df_diamonds_train.groupby(['cut'])['price'].mean().to_dict()
X_test['cut_encoding'] = X_test['cut'].map(cut_encoding).astype(float)
color_encoding = df_diamonds_train.groupby(['color'])['price'].mean().to_dict()
X_test['color_encoding'] = X_test['color'].map(color_encoding).astype(float)
clarity_encoding = df_diamonds_train.groupby(['clarity'])['price'].mean().to_dict()
X_test['clarity_encoding'] = X_test['clarity'].map(clarity_encoding).astype(float)
# Std
cut_encoding_std = df_diamonds_train.groupby(['cut'])['price'].std().to_dict()
X_test['cut_encoding_std'] = X_test['cut'].map(cut_encoding_std).astype(float)
color_encoding_std = df_diamonds_train.groupby(['color'])['price'].std().to_dict()
X_test['color_encoding_std'] = X_test['color'].map(color_encoding_std).astype(float)
clarity_encoding_std = df_diamonds_train.groupby(['clarity'])['price'].std().to_dict()
X_test['clarity_encoding_std'] = X_test['clarity'].map(clarity_encoding_std).astype(float)

### 5.2 Including volume

In [29]:
#X_test['volume']=X_test['x']*X_test['y']*X_test['z']

### 5.3 Cross Target encoding (mean and std)

In [30]:
# 2. Cross target encoding
# Mean
cut_color_encoding = df_diamonds_train.groupby(['cut','color'])['price'].mean().to_dict()
X_test['cut_color_encoding'] = X_test.set_index(['cut','color']).index.map(cut_color_encoding.get).astype(float)
cut_clarity_encoding = df_diamonds_train.groupby(['cut','clarity'])['price'].mean().to_dict()
X_test['cut_clarity_encoding'] = X_test.set_index(['cut','clarity']).index.map(cut_clarity_encoding.get).astype(float)
color_clarity_encoding = df_diamonds_train.groupby(['color','clarity'])['price'].mean().to_dict()
X_test['color_clarity_encoding'] = X_test.set_index(['color','clarity']).index.map(color_clarity_encoding.get).astype(float)

# Std  
#cut_color_encoding_std = df_diamonds_train.groupby(['cut','color'])['price'].std().to_dict()
#X_test['cut_color_encoding_std'] = X_test.set_index(['cut','color']).index.map(cut_color_encoding_std.get).astype(float)
#cut_clarity_encoding_std = df_diamonds_train.groupby(['cut','clarity'])['price'].std().to_dict()
#X_test['cut_clarity_encoding_std'] = X_test.set_index(['cut','clarity']).index.map(cut_clarity_encoding_std.get).astype(float)
#color_clarity_encoding_std = df_diamonds_train.groupby(['color','clarity'])['price'].std().to_dict()
#X_test['color_clarity_encoding_std'] = X_test.set_index(['color','clarity']).index.map(color_clarity_encoding_std.get).astype(float)

### 5.4 Feature Scaling

In [31]:
#X_test_num=X_test[num_features_list]
#X_test_carat=X_test[cat_features_list]

In [32]:
# Perform the feature scaling on the numeric attributes of the dataset
#num_scaler = StandardScaler()
#X_test_num_scaled = num_scaler.fit_transform(X_test_num)
#X_test_num_scaled=pd.DataFrame(X_test_num_scaled)
#X_test_num_scaled.rename(columns = {0:'x', 1:'y', 2:'z', 3:'depth', 4:'table', 5:'carat'}, 
#                           inplace = True)
#X_test_num_scaled.head()

In [33]:
#X_test=pd.merge(X_test_num_scaled, X_test_carat, left_index=True, right_index=True)
#X_test.head()

### 5.5 One hot encoding

In [34]:
# 3. One-hot encoding for categorical variables
X_test=pd.get_dummies(X_test,columns=cat_features_list,drop_first=True)

In [35]:
X_test.shape

(13485, 32)

In [36]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   x                       13485 non-null  float64
 1   y                       13485 non-null  float64
 2   z                       13485 non-null  float64
 3   depth                   13485 non-null  float64
 4   table                   13485 non-null  float64
 5   carat                   13485 non-null  float64
 6   cut_encoding            13485 non-null  float64
 7   color_encoding          13485 non-null  float64
 8   clarity_encoding        13485 non-null  float64
 9   cut_encoding_std        13485 non-null  float64
 10  color_encoding_std      13485 non-null  float64
 11  clarity_encoding_std    13485 non-null  float64
 12  cut_color_encoding      13485 non-null  float64
 13  cut_clarity_encoding    13485 non-null  float64
 14  color_clarity_encoding  13485 non-null

## Model definition - RandomForestRegressor - with Random Hyperparameter Grid

RandomForestRegressor: multiple trees in paralel changing samples and convining diferrent features (overfitting when the tree is big and good to reduce error variance)

Main Parameters:
   - bootstrap -> method for sampling data points (TRUE bagging and FALSE pasting, with/without replacement)
   - n_estimators -> number of trees in the foreset
   - max_depth -> max number of levels in each decision tree
   - max_features -> max number of features considered for splitting a node
   - ccp_alpha ->
   - criterion ->
   - max_leaf_nodes -> max number of solution nodes 
   - max_samples ->
   - min_impurity_decrease ->
   - min_samples_leaf -> min number of data points allowed in a leaf node
   - min_samples_split -> min number of data points placed in a node before the node is split
   - min_weight_fraction_leaf
   - n_estimators -> number of trees in the foreset
   - n_jobs 
   - oob_score 
   - random_state
   - verbose
   - warm_start   

In [37]:
# 0. Random Hyperparameter Grid - Grid definition

#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] # Number of trees in random forest
#max_features = ['auto', 'sqrt'] # Number of features to consider at every split
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Maximum number of levels in tree
#max_depth.append(None)
#min_samples_split = [2, 5, 10] # Minimum number of samples required to split a node
#min_samples_leaf = [1, 2, 4] # Minimum number of samples required at each leaf node
#bootstrap = [True, False] # Method of selecting samples for training each tree

# Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}
#print(random_grid)

In [38]:
#%%time
#1. Random Hyperparameter Grid - Use the random grid to search for best hyperparameters

# First create the base model to tune
#rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
#search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100,
#                               cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
#rf_random.fit([(X_train,y_train),(X_val,y_val)])

In [39]:
#rf_random.best_params_

In [40]:
#best_params={'n_estimators': 1600,
# 'min_samples_split': 5,
# 'min_samples_leaf': 1,
# 'max_features': 'auto',
# 'max_depth': 90,
# 'bootstrap': True}

#(n_estimators=100,min_samples_split=5,min_samples_leaf=1,
#                              max_features='auto',max_depth=10,bootstrap=True)

#n_estimators=100,min_samples_split=5,min_samples_leaf=1,
#                              max_features='auto',max_depth=40,bootstrap=True

In [41]:
# 2. RandomForestRegressor definition
model = RandomForestRegressor(n_estimators=500,min_samples_split=5,min_samples_leaf=1,
                              max_features='auto',max_depth=10,bootstrap=True, random_state=42)

In [42]:
#Importance
#feats = {} # a dict to hold feature_name: feature_importance
#for feature, importance in zip(X_train.columns, model.feature_importances_):
#    feats[feature] = importance #add the name/value pair 

In [43]:
#Importance=pd.DataFrame(list(feats.items()),columns = ['feature','importance']).sort_values('importance',ascending=False)

In [44]:
# 1. XGBRegressor 
model = XGBRegressor(n_estimators=200,colsample_bylevel=1,colsample_bynode=1,
                     colsample_bytree=0.8,reg_alpha=1, reg_lambda=1,gamma=0,learning_rate=0.1, random_state=42)
hyperparameters = model.get_params()
print(type(model), '\n')
print('Model hyperparameters:', hyperparameters, '\n')
#n_estimators=200,colsample_bylevel=1,colsample_bynode=1,
#                     colsample_bytree=0.8,reg_alpha=1, reg_lambda=1,gamma=0,learning_rate=0.1, random_state=42

<class 'xgboost.sklearn.XGBRegressor'> 

Model hyperparameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.8, 'enable_categorical': False, 'gamma': 0, 'gpu_id': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_delta_step': None, 'max_depth': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 200, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': 42, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None} 



In [45]:
#model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
#                    metric_params=None, n_jobs=None, n_neighbors=4, p=2,
#                    weights='uniform')

## Model training with validation

In [46]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [47]:
%%time
# 1. XGBRegressor early_stopping_rounds=40
# Model training
model.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=10)

[0]	validation_0-rmse:4596.78125	validation_1-rmse:4444.73779
[1]	validation_0-rmse:4154.49609	validation_1-rmse:4015.98608
[2]	validation_0-rmse:3755.30322	validation_1-rmse:3629.67700
[3]	validation_0-rmse:3397.01367	validation_1-rmse:3282.51172
[4]	validation_0-rmse:3074.68042	validation_1-rmse:2971.11914
[5]	validation_0-rmse:2782.96704	validation_1-rmse:2686.63623
[6]	validation_0-rmse:2521.97266	validation_1-rmse:2432.28271
[7]	validation_0-rmse:2289.06226	validation_1-rmse:2207.06519
[8]	validation_0-rmse:2079.49585	validation_1-rmse:2004.06787
[9]	validation_0-rmse:1891.00256	validation_1-rmse:1820.07593
[10]	validation_0-rmse:1720.88977	validation_1-rmse:1655.27881
[11]	validation_0-rmse:1570.47131	validation_1-rmse:1509.14490
[12]	validation_0-rmse:1436.91235	validation_1-rmse:1379.96509
[13]	validation_0-rmse:1314.11987	validation_1-rmse:1260.73547
[14]	validation_0-rmse:1207.24341	validation_1-rmse:1158.12659
[15]	validation_0-rmse:1110.01904	validation_1-rmse:1064.58887
[1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [51]:
%%time
# Model predictions
y_pred_val = model.predict(X_val)

CPU times: user 45.9 ms, sys: 3.51 ms, total: 49.4 ms
Wall time: 10.7 ms


In [52]:
%%time
# Model predictions
y_pred_train = model.predict(X_train)

CPU times: user 157 ms, sys: 4.78 ms, total: 161 ms
Wall time: 31.8 ms


## Training set error

In [53]:
# Model predictions
rmse_train = mean_squared_error(y_train, y_pred_train)**0.5
rmse_train

375.19003898544906

In [54]:
mae_train=mean_absolute_error(y_train, y_pred_train)
mae_train

213.54753637593763

In [55]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9842300177530219

## Model validation

In [56]:
#432
rmse_val = mean_squared_error(y_val, y_pred_val)**0.5
rmse_val

434.48087014011406

In [57]:
mae_val=mean_absolute_error(y_val, y_pred_val)
mae_val

234.71701912961197

In [58]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9842300177530219

## Model training without validation

In [59]:
%%time
# Model training
model.fit(X, y,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:4597.02295	validation_1-rmse:4444.88477
[1]	validation_0-rmse:4154.59912	validation_1-rmse:4016.24658
[2]	validation_0-rmse:3755.22803	validation_1-rmse:3629.36987
[3]	validation_0-rmse:3396.84082	validation_1-rmse:3281.95801
[4]	validation_0-rmse:3074.45190	validation_1-rmse:2967.44580
[5]	validation_0-rmse:2782.80688	validation_1-rmse:2683.27539
[6]	validation_0-rmse:2522.51831	validation_1-rmse:2429.31885
[7]	validation_0-rmse:2289.94995	validation_1-rmse:2204.10742
[8]	validation_0-rmse:2080.95093	validation_1-rmse:2000.82190
[9]	validation_0-rmse:1893.23071	validation_1-rmse:1818.09277
[10]	validation_0-rmse:1723.51709	validation_1-rmse:1653.19983
[11]	validation_0-rmse:1573.46399	validation_1-rmse:1507.18286
[12]	validation_0-rmse:1439.83728	validation_1-rmse:1377.41663
[13]	validation_0-rmse:1317.27954	validation_1-rmse:1258.04004
[14]	validation_0-rmse:1211.75708	validation_1-rmse:1154.45898
[15]	validation_0-rmse:1114.75037	validation_1-rmse:1060.32361
[1

[134]	validation_0-rmse:365.51865	validation_1-rmse:353.97308
[135]	validation_0-rmse:364.68961	validation_1-rmse:353.16693
[136]	validation_0-rmse:364.46765	validation_1-rmse:353.08038
[137]	validation_0-rmse:364.28143	validation_1-rmse:352.88367
[138]	validation_0-rmse:364.07288	validation_1-rmse:352.71335
[139]	validation_0-rmse:363.37628	validation_1-rmse:352.08942
[140]	validation_0-rmse:362.95841	validation_1-rmse:351.64874
[141]	validation_0-rmse:362.40787	validation_1-rmse:351.07684
[142]	validation_0-rmse:361.83551	validation_1-rmse:350.84750
[143]	validation_0-rmse:360.99509	validation_1-rmse:350.46939
[144]	validation_0-rmse:360.31168	validation_1-rmse:349.85126
[145]	validation_0-rmse:360.23386	validation_1-rmse:349.82138
[146]	validation_0-rmse:359.65131	validation_1-rmse:349.24103
[147]	validation_0-rmse:359.37582	validation_1-rmse:349.13065
[148]	validation_0-rmse:358.77728	validation_1-rmse:348.75995
[149]	validation_0-rmse:358.61395	validation_1-rmse:348.63501
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Test Preditions

In [60]:
predictions = model.predict(X_test)

In [61]:
predictions=pd.DataFrame(predictions)

In [62]:
predictions.reset_index(inplace=True)

In [63]:
predictions=predictions.rename({0: 'price','index': 'id'}, axis=1)

## Save Preditions

In [64]:
predictions.to_csv('../data/diamonds_predictions_XGBRegressor_Cross_Target.csv',index=False)